In [1]:
from genefab import get_datasets, GLDS

In [2]:
datasets = get_datasets(
    ptype="flight", organism="mus", factor="radiation", assay="transcript",
    maxcount=3
)

looking up ptype(s): "Spaceflight Study", "Spaceflight Project", "Spaceflight", "Flight Study", "Flight",
looking up organism(s): "Mus musculus",
looking up factor(s): "Absorbed Radiation Dose", "Ionizing Radiation", "Ionzing Radiation", "Irradiation", "post radiation timepoint", "Radiation", "Radiation Distance", "Radiation dosage", "radiation dose", "radiation type", "Radiation, Ionzing",
looking up assay(s): "transcription profiling",
Parsing url:  https://genelab-data.ndc.nasa.gov/genelab/data/search/?term=GLDS&type=cgene&size=3&ffield=Project+Type&fvalue=Spaceflight+Study&ffield=Project+Type&fvalue=Spaceflight+Project&ffield=Project+Type&fvalue=Spaceflight&ffield=Project+Type&fvalue=Flight+Study&ffield=Project+Type&fvalue=Flight&ffield=organism&fvalue=Mus+musculus&ffield=Study+Factor+Name&fvalue=Absorbed+Radiation+Dose&ffield=Study+Factor+Name&fvalue=Ionizing+Radiation&ffield=Study+Factor+Name&fvalue=Ionzing+Radiation&ffield=Study+Factor+Name&fvalue=Irradiation&ffield=Study+Factor

In [3]:
datasets

[GLDS-87 (number of assays: 1; factors: Spaceflight, Absorbed Radiation Dose),
 GLDS-173 (number of assays: 1; factors: Space Flight, Absorbed Radiation Dose),
 GLDS-25 (number of assays: 1; factors: Space Flight, Absorbed Radiation Dose)]

In [4]:
glds = GLDS("GLDS-116")

Parsing url:  https://genelab-data.ndc.nasa.gov/genelab/data/study/data/GLDS-116/
Parsing url:  https://genelab-data.ndc.nasa.gov/genelab/data/glds/files/116


In [5]:
glds.factors

['Microgravity', 'Absorbed Radiation Dose']

In [6]:
len(glds.assays)

2

In [7]:
assay = glds.assays[0]
assay.dataframe

,Comment: Data Analysis Report 1,Protocol REF,Hybridization Assay Name,Array Data File,Protocol REF,Comment: Data Analysis Report 2,Comment: Derived Array Data FIle 2,Comment: Array Data File 2,Comment: Group Name,Protocol REF,Extract Name,Protocol REF,Sample Name,Parameter Value: Array Design REF,Factor Value: Microgravity,Protocol REF,Labeled Extract Name,Derived Array Data File,Parameter Value: Hybridization Assay Name 2,Array Design REF
0,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Control Group,nucleic acid hybridization,A32 Extract,data collection,A32,PAMM-065A,Ground Control,labeling,A32 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
1,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Control Group,nucleic acid hybridization,A34 Extract,data collection,A34,PAMM-065A,Ground Control,labeling,A34 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
2,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Control Group,nucleic acid hybridization,A36 Extract,data collection,A36,PAMM-065A,Ground Control,labeling,A36 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
3,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Control Group,nucleic acid hybridization,A38 Extract,data collection,A38,PAMM-065A,Ground Control,labeling,A38 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
4,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Control Group,nucleic acid hybridization,A40 Extract,data collection,A40,PAMM-065A,Ground Control,labeling,A40 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
5,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Control Group,nucleic acid hybridization,A42 Extract,data collection,A42,PAMM-065A,Ground Control,labeling,A42 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
6,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PAMM-013.csv,RNA extraction,*PCR Array Analysis PAMM-065.pdf,*PCR Array Analysis PAMM-065_derived.csv,*PCR Array Analysis PAMM-065.csv,Group 1,nucleic acid hybridization,F52 Extract,data collection,F52,PAMM-065A,Spaceflight,labeling,F52 LE,*PCR Array Analysis PAMM-013_derived.csv,Mouse Oxidative Stress and Antioxidant Defense...,PAMM-013A
7,*PCR Array Analysis PAMM-013.pdf,normalization data transformation,Mouse Extracellular Matrix and Adhesion Molecu...,*PCR Array Analysis PA

In [8]:
assay.get_file_url("*PCR Array Analysis PAMM-013.csv")

'https://genelab-data.ndc.nasa.gov/datamanager/file/Home/Public/genelab/GLDS-116/microarray/GLDS-116_microarray_PCR Array Analysis PAMM-013.csv'